In [ ]:
import openpyxl
import pandas as pd
import glob
import os
import re

summary_dir=r'C:\Users\fbpza\Desktop\My_utils\Summary files'
BOQ_file_path=r'C:\Users\fbpza\Desktop\My_utils\Peshawar Highway 02 - BoQ.xlsx'
output_file_path = BOQ_file_path

In [ ]:
def format_segment_name(segment_name):
    match = re.match(r'Segment (\d+)([A-Z]*)', segment_name)
    if match:
        number_part = match.group(1)
        alpha_part = match.group(2)
        formatted_number_part = str(int(number_part))
        return formatted_number_part + alpha_part
    return segment_name  # Return as is if no match

In [ ]:
summ_files = glob.glob(os.path.join(summary_dir, '*.csv'))
roads_dict = {}

for summ_file in summ_files:
    road_name = os.path.basename(summ_file).split('_Summary')[0]
    summ_df = pd.read_csv(summ_file)
    
    # Create a dictionary for segments with PCI and Width values
    segments_dict = summ_df.set_index('Segments')[['PCI', 'Width']].to_dict(orient='index')
    
    # Create a formatted dictionary with the required structure
    formatted_segments_dict = {format_segment_name(seg): {'PCI': data['PCI'], 'Width': data['Width']} for seg, data in segments_dict.items()}
    
    roads_dict[road_name] = formatted_segments_dict

boq_workbook = openpyxl.load_workbook(BOQ_file_path)
BOQ_DF = pd.ExcelFile(BOQ_file_path)
boq_sheets = BOQ_DF.sheet_names


for sheet in boq_sheets:
    print(f"Processing sheet: {sheet}")

    cost_df = pd.read_excel(BOQ_file_path, sheet_name=sheet, header=2)
    cost_df_header0 = pd.read_excel(BOQ_file_path, sheet_name=sheet, header=0)
    print()
    
    # Check if sheet name should be updated
    if sheet != cost_df_header0.columns[0]:
        # Rename the sheet
        sheet_obj = boq_workbook[sheet]
        new_title = cost_df_header0.columns[0]
        print(f"Renaming sheet from '{sheet}' to '{new_title}'")
        sheet_obj.title = new_title

    rd_name_inBOQ = cost_df_header0.columns[0]
    S_no = cost_df['S/No'].tolist()
    PCI_column = cost_df['PCI Value / Road Classification']
    length_in_meters = cost_df['Length'].tolist()

    # Replace "Asphalt" with the PCI value
    updated_pci_column = []
    updated_width_column = []
    
    
    for each_segNo in S_no:
        segindex  = S_no.index(each_segNo)
        pci_value = PCI_column[segindex]
        each_segNo= str(each_segNo)
        
        width_value='- -'
        
        if pci_value == 'Asphalt':
            seg_number=each_segNo.lstrip()
            seg_number=re.sub(r'^0+', '', seg_number)
            seg_full_name = seg_number
            
            if rd_name_inBOQ in roads_dict and seg_full_name in roads_dict[rd_name_inBOQ]:
                pci_value = roads_dict[rd_name_inBOQ][seg_full_name]['PCI']
                width_value = roads_dict[rd_name_inBOQ][seg_full_name]['Width']
            else:
                print(f"Segment {seg_full_name} not found in roads_dict or has no PCI value.")
        
        updated_pci_column.append(pci_value)
        updated_width_column.append(width_value)
    
    # Update the DataFrame with the new PCI values
    cost_df['PCI Value / Road Classification'] = updated_pci_column
    cost_df['Width'] = updated_width_column
    # cost_df_header0.append(cost_df)
    # cost_df = cost_df_header0
    
    # df11=cost_df.astype(str)
    # df22=cost_df_header0.astype(str)
    # combined_df=pd.concat([df11,df22],axis=1)
    # cost_df=combined_df
    # Save the updated DataFrame back to the Excel sheet
    with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        cost_df.to_excel(writer, sheet_name=sheet, index=False)

print("Processing complete.")

